<a href="https://colab.research.google.com/github/argonism/DEIM2024_hands-on/blob/main/DEIM2024_%E5%A4%A7%E8%A6%8F%E6%A8%A1%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%86%E3%82%99%E3%83%AB%E3%81%AB%E5%9F%BA%E3%81%A4%E3%82%99%E3%81%8F%E6%A4%9C%E7%B4%A2%E3%83%A2%E3%83%86%E3%82%99%E3%83%AB%5BTU_C_2_%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
# DEIM 2024 チュートリアル **大規模言語モデルに基づく検索モデル** \[TU-C-2\]

## 概要
近年の大規模言語モデルに基づく検索モデルを用いた検索実験のデモ

DPRの学習と評価のデモを通して、よく用いられているフレームワークやツールを示します．

ここでは，nfcorpusというデータセットを用いて，DPRというBERTベースの密検索モデルを訓練します．
1. ファインチューニングする前に、DPRのnfcorpusでの性能を評価します．
1. nfcorpusの訓練セットでDPRを訓練します．
1. 2.で得られたDPRを再びnfcorpusで評価し，DPRのin-domainの検索性能を確認します．

## 始める前に
**ランタイムのタイプがGPU（e.g. T4 GPU）になっていることを確認してください！**

確認方法
- colab上部のナビゲーションバーから「ランタイム」> 「ランタイムのタイプを変更」
- 「ハードウェア アクセラレータ」の指定がGPUになっていることを確認して、閉じる。
- なっていなければ選択できるGPUを指定して「保存」を押す

## 依存関係のインストール

今回は、DPRのファインチューニングにTevatronという大規模言語モデルの訓練ツールキットを使います。

そのため、まずはTevatronと、依存パッケージをインストールします。

In [ ]:
import time; COLAB_START_TIME = time.time()

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2024-02-08, rust version 1.76.0 (07dca489a 2024-02-04)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
  8.5 MiB /   8.5 MiB (100 %)   6.5 MiB/s in  1s ETA:  0s
info: installing component 'clippy'
info: installing component 'rust-docs'
 14.7 MiB /  14.7 MiB (100 %)   1.0 MiB/s in 11s ETA:  0s
info: installing component 'rust-std'
 23.9 MiB /  23.9 MiB (100 %)   7.9 MiB/s in  3s ETA:  0s
info: installing component 'rustc'
 62.3 MiB /  62.3 MiB (100 %)  11.5 MiB/s in  6s ETA:  0s
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu

### **Tevatron**

大規模言語モデルに基づく検索モデルの学習や評価に焦点を当てたpythonフレームワーク

情報検索系のフレームワークの中では大規模言語モデル系の検索モデルの学習に強いという特徴があります．

### その他のIRツール
- Ancerini・Pyserini
  - 検索ツールキットで，簡単に論文の結果を再現するというところに焦点を当てている．PyseriniはAnceriniのpythonバインディング．
- Terrier・Pyterrier
  - 拡張性・柔軟性が高い情報検索フレームワークで、最近の大規模言語モデルに基づく検索モデルの実装もプラグインとして公開されている。Pyterrierはterrierのpythonバインディング

In [ ]:
!pip install git+https://github.com/texttron/tevatron
!pip install git+https://github.com/luyug/GradCache
!pip install torch==1.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!PATH="$HOME/.cargo/bin:$PATH" pip install -U faiss-cpu==1.7.2 transformers==4.16.0 datasets==1.17.0

  Cloning https://github.com/texttron/tevatron to /tmp/pip-req-build-ewl7kzz3
  Running command git clone --filter=blob:none --quiet https://github.com/texttron/tevatron /tmp/pip-req-build-ewl7kzz3
  Resolved https://github.com/texttron/tevatron to commit 2e5d00ee21d5a7db0bd2ea1463c9150a572106d4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
  Created wheel for tevatron: filename=tevatron-0.0.1-py3-none-any.whl size=38500 sha256=317516cfecd9303df4db226444123e461f143a0725d1fdf81792faef8b0e9a55
  Stored in directory: /tmp/pip-ephem-wheel-cache-44c0mn1j/wheels/bc/1e/c8/3bc95e9e1ca1ebe94b78ca794b9a19de36adf4c6faf86b2346
Successfully built tevatron
  Cloning https://github.com/luyug/GradCache to /tmp/pip-req-build-x2bqs9gk
  Running command git clone --fi

こちらは評価に使うパッケージと、評価スクリプトのコードを使いたいので、Tevatronのソースコードをcloneします。

In [ ]:
!pip install pyserini
!git clone https://github.com/texttron/tevatron.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.1 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
  Created wh

## DPRの現時点の検索性能

msmarco-passageでfine-tuningされたDPRを、nfcorpusでファインチューニングしていきます。

その前に、今の時点ではどのくらいの性能が出ているのか見てみましょう。

### Tevatronのスクリプトを用いて、nfcorpusで評価

nfcorpusはBEIRというベンチマークに含まれているデータセットの一つです。

Tevatronには検索モデルをBEIRで簡単に評価できるようなスクリプトがあるので使わせてもらいましょう。

In [ ]:
%env BASE_IR_MODEL=k-ush/tevatron_dpr

env: BASE_IR_MODEL=k-ush/tevatron_dpr


In [ ]:
!mkdir beir_embedding_k-ush
!bash tevatron/scripts/eval_beir.sh $BASE_IR_MODEL nfcorpus

2024-03-01 11:46:33.574172: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 11:46:33.574229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 11:46:33.578707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 11:46:33.589802: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 11:46:35.308756: W tensorflow/compiler/tf2

## DPRを訓練する

それでは、DPRを訓練していきます。

引き続きTevatronを使って、nfcorpusの訓練データセットでDPRを訓練していきますが、その前にnfcorpusの訓練データをTevatronで扱えるフォーマットに直す必要があります。

### 訓練データの整形
Tevatronでの訓練データセットの形はDPRで使われる訓練データセットと似ており，以下の形式のjsonで構成されるjsonlで食わせます．内部的にはdatasetsのload_datasetを読んでいるので，load_datasetで読んで下記の構造のデータになればなんでも良いです．

```
{
  query_id: ...,
  query: ...,
  positive_passages: [
    {
      docid: ...,
      title: ...,
      text: ...,
    }
  ],
  negative_passages: [
    {
      docid: ...,
      title: ...,
      text: ...,
    }
  ]
}
```

訓練データは，情報検索向けのデータセットを集めて，同じようなインターフェースからアクセスできるようにしているライブラリである[`ir-datasets`](https://ir-datasets.com/)を用いる．

In [ ]:
!pip install ir_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=8f129ac9271589c2b1384970a403b4a8df3c6f3fdf4f0cdbc49d553a9db5a6e7
  Stored in directory: /root/.cache/pip/wheels/1a/d7/91/7ffb991df87e62355d945745035470ba2616aa3d83a250b5f9
  Created wheel for zlib-state: filename=zlib_stat

In [ ]:
from collections.abc import Callable
from pathlib import Path
from typing import Union, Dict, List, Set, Callable
from collections import defaultdict, namedtuple
import random
import json

import ir_datasets as irds
from tqdm import tqdm

Qrels = Dict[str, Dict[str, str]]

class IrdsToTevatronDataset(object):
  def __init__(self, dataset_key: str) -> None:
    self.dataset = self._load_dataset(dataset_key)

  def _load_dataset(self, dataset_key: str) -> irds.Dataset:
    return irds.load(dataset_key)

  def load_doc_ids(self) -> List[str]:
    doc_ids = []
    for doc in tqdm(self.dataset.docs_iter(), total=self.dataset.docs_count(), desc="loading doc id"):
      doc_ids.append(doc.doc_id)
    return doc_ids

  def load_query_table(self, query_field: str = "text") -> Dict[str, str]:
    queries = {}
    for query in tqdm(self.dataset.queries_iter(), total=self.dataset.queries_count(), desc="loading query"):
      queries[query.query_id] = getattr(query, query_field)
    return queries

  def load_qrels_table(self) -> Qrels:
    qrels = defaultdict(dict)
    for qrel in tqdm(self.dataset.qrels_iter(), total=self.dataset.qrels_count(), desc="loading qrels"):
      qrels[qrel.query_id][qrel.doc_id] = qrel.relevance
    return qrels

  def sample_random_negatives(self, doc_ids: List[str], exclude_doc_ids: Union[Set[str], List[str]], k: int = 1) -> List[str]:
    exclude_doc_ids = set(exclude_doc_ids)
    sample_source = [doc_id for doc_id in doc_ids if not doc_id in exclude_doc_ids]
    negative_doc_ids = random.choices(sample_source, k=k)
    return random.choices(sample_source, k=k)

  def prepare_train_dataset(self, output_path: Union[str, Path], doc_preprocess: Callable[[namedtuple], str], queries_num: int = 500):
    output_path = Path(output_path)

    docstore = self.dataset.docs_store()
    doc_ids = self.load_doc_ids()
    queries = self.load_query_table()
    qrels = self.load_qrels_table()

    with output_path.open("w") as fw:
      total = min(len(queries), queries_num)
      for i, qid in enumerate(tqdm(queries, desc="writing train dataset", total=total)):
        if i >= total: break

        query = queries[qid]
        relevant_doc_ids = [doc_id for doc_id in qrels[qid].keys() if qrels[qid][doc_id] > 0]
        negative_ids = self.sample_random_negatives(doc_ids, relevant_doc_ids, k=len(relevant_doc_ids))
        negatives = [doc_preprocess(doc) for docid, doc in docstore.get_many(negative_ids).items()]
        positives = [doc_preprocess(doc) for docid, doc in docstore.get_many(relevant_doc_ids).items()]
        train_json = {
            "query_id": qid,
            "query": query,
            "positive_passages": positives,
            "negative_passages": negatives
        }
        fw.write(json.dumps(train_json, ensure_ascii=False) + "\n")

dataset_key = "beir/nfcorpus/train"
dataset_path = "/content/nfcorpus.train.jsonl"
queries_num = 1000

def nfcorpus_doc_preprocess(doc: namedtuple) -> str:
  return {
      "docid": doc.doc_id,
      "title": doc.title,
      "text": doc.text,
  }

irds_to_tev = IrdsToTevatronDataset(dataset_key)
irds_to_tev.prepare_train_dataset(dataset_path, doc_preprocess=nfcorpus_doc_preprocess, queries_num=queries_num)

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip
docs_iter:   0%|                                      | 0/3633 [00:01<?, ?doc/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip: 0.0%| 0.00/2.45M [00:00<?, ?B/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip: 2.0%| 49.2k/2.45M [00:00<00:07, 300kB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip: 4.7%| 115k/2.45M [00:00<00:05, 417kB/s] 
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip: 10.7%| 262k/2.45M [00:00<00:03, 630kB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip: 22.1%| 541k/2.45M [00:00<00:01, 961kB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip: 45.5%| 1.11M/2.45M [00:00<00:00, 1.59MB/s]

[INFO] [finished] https

### **訓練の実行**

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python -m tevatron.driver.train \
  --output_dir dpr_nfcorpus \
  --model_name_or_path $BASE_IR_MODEL \
  --save_steps 10 \
  --dataset_name Tevatron/wikipedia-nq \
  --train_dir /content/nfcorpus.train.jsonl \
  --fp16 \
  --per_device_train_batch_size 128 \
  --positive_passage_no_shuffle \
  --train_n_passages 2 \
  --learning_rate 1e-5 \
  --q_max_len 32 \
  --p_max_len 156 \
  --num_train_epochs 5 \
  --logging_steps 500 \
  --grad_cache \
  --overwrite_output_dir

2024-03-01 11:50:51.491266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 11:50:51.491328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 11:50:51.493133: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 11:50:51.503318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 11:50:53.089354: W tensorflow/compiler/tf2

## 評価

In [ ]:
!bash tevatron/scripts/eval_beir.sh dpr_nfcorpus nfcorpus

In [ ]:
COLAB_END_TIME = time.time()

In [ ]:
f"{(COLAB_END_TIME - COLAB_START_TIME) / 60:.2f}"